# Sistema de recomendación de anime

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

#Barra de progreso
import time
from tqdm import tqdm


El objetivo es aplicar todo lo que hemos aprendido hasta el momento, especialmente sobre gradiente descendiente. Para esto desarrollaremos un sistema de recomendación usando factorización de matrices.

En esta entrega se darán todos elementos necesarios para desarrollar un sistema de recomendación de amines, el objetivo es que usted entienda como funciona y desarrolle una solución.

Si desea conocer mas sobre el dataset que usaremos puede revisar acá: https://www.kaggle.com/CooperUnion/anime-recommendations-database

1- Cargue las bases de datos de calificaciones, usuarios y animes. Puede usar para este objetivo la biblioteca pandas. Genere el conjunto de entrenamiento usando los usuarios cómo filas y los animes cómo columnas. Tenga en cuenta que para el algoritmo es necesario que los -1 sean cero, sin embargo, para la recomendación si debe tener en cuenta dichos valores.

In [12]:
#Obtener Listado .CSV
def cargarAnimes():
    df = pd.read_csv('anime.csv', delimiter=',', parse_dates=[6], header=0,index_col=False, squeeze=False)
    return np.array(df)

#Números globales cantidad animes y usuarios
cantAnimes=12294
cantUsers=73516

#Crear R a partir del archivo csv
def crearR(ceros):
    aux=-1
    if ceros:
        aux=0
    uf = pd.read_csv('rating.csv', delimiter=',', parse_dates=[2], header=0,index_col=False, squeeze=False)
    u=np.array(uf)
    
    #Crear R
    R = np.zeros((cantUsers,cantAnimes), dtype = np.int8)
    for i in u:
        if i[1] <= cantAnimes and int(i[2])>=aux:
            R[i[0]-1,i[1]-1] = int(i[2])
      
    return R 


2- Agregue la función donde se ejecuta el algoritmo de factorización de matrices. Puede basarse en el algoritmo compartido en las dispositivas. No debe de gastar todas las iteraciones. 

In [3]:
def matrix_factorization(R, P, Q, K, steps=500, alpha=0.01, beta=0.001):
    Q = Q.T
    error=0
    for step in tqdm(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        aux1=(2 * eij * Q[k][j] - beta * P[i][k])
                        aux2=(2 * eij * P[i][k] - beta * Q[k][j])
                        P[i][k] = P[i][k] + alpha * aux1
                        Q[k][j] = Q[k][j] + alpha * aux2
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        error=e
        if e < 0.5:
            break
    print('Error',error)
    return P, Q.T

3- Programe una función para generar $\hat{R}$. Recuerde que para generar $\hat{R}$ debe realizar los siguiente pasos:

    - Generar R usando el algoritmo de vecinos más cercanos.
    - Defina K.
    - Defina P y Q.
    - Generar R prima.


In [4]:
def guardar(name,matriz):
    np.savetxt(name,matriz)

def cargar(name):
    return np.loadtxt(name)

def generateRandomPQ(k):
    Q = np.random.random((cantAnimes, k)).astype(np.float32)
    P = np.random.random((cantUsers, k)).astype(np.float32)
    return P,Q

def generateR(usuario, vecinos):
    R = crearR(True)
    RSelected = np.empty([vecinos,cantAnimes])
    distances,indices=knn(True,usuario,vecinos,R)
    for i in range(0, len(indices[0])-1, 1):
        RSelected[i] = R[indices[0,i]]
    del R
    return RSelected

def knn(cargado,usuario,vecinos,R):
    if not(cargado):
        nbrs = NearestNeighbors(n_neighbors=vecinos, algorithm='auto', n_jobs=-1).fit(R)
        distances, indices = nbrs.kneighbors(R[usuario-1].reshape((1, -1)))
        return distances,indices
    else:
        if vecinos>30:
            vecinos=30
        if usuario in [51,510,17000,27005,37502,57502,67501]:
            distances=np.loadtxt('distances'+str(usuario)+'.txt')
            indices=np.loadtxt('indices'+str(usuario)+'.txt')
            return distances[:vecinos].reshape(1,vecinos),indices[:vecinos].reshape(1,vecinos).astype(int)
        else:
            distances=np.loadtxt('distances.txt')
            indices=np.loadtxt('indices.txt')
            return distances.reshape(1,vecinos),indices.reshape(1,vecinos).astype(int)
        
def optimizarMatriz(R,indices,distances):
    userIguales=np.array([]).reshape(0,2)
    cont=0
    pos=-1
    Raux=np.zeros((len(R),len(R[0])),dtype=np.int8)
    for i in tqdm(range(len(indices))):
        for j in range(len(indices[0])):
            if int(distances[i,j]) == 0:
                index=int(indices[i,j])
                aux1=np.where(userIguales[:,1]==i)
                if i != index:
                    aux2=np.where(userIguales[:,1]==index)
                    if len(aux1[0]) == 0:
                        if len(aux2[0]) == 0:
                            pos=pos+1
                            userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,i]), 0)
                            userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,index]), 0)
                            Raux[cont]=R[i]
                            cont=cont+1
                        else:
                            userIguales=np.insert(userIguales,aux2[0][-1]+1, np.array([userIguales[aux2[0][-1],0],i]), 0)
                    else:
                        if len(aux2[0]) == 0:
                            userIguales=np.insert(userIguales,aux1[0][-1]+1, np.array([userIguales[aux1[0][-1],0],index]), 0)                  
                else:
                    if len(aux1[0]) == 0:
                        pos=pos+1
                        userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,i]), 0)
                        Raux[cont]=R[i]
                        cont=cont+1
    return Raux[:cont,:], userIguales

def generateRPrime(k, usuario, vecinos):
    P,Q = generateRandomPQ(k)
    RSelected = generateR(usuario, vecinos)
    P,Q = matrix_factorization(RSelected, P, Q, k)
    return RSelected, np.dot(P, Q.T)

def obtenerRPrima(cargado,usuario,kNeighbors,kFactorizacion):
    if not(cargado) or not(usuario in [51,510,17000,27005,37502,57502,67501]):
        Rl, Rp = generateRPrime(kFactorizacion, usuario, kNeighbors)
        return Rl,Rp
    else:
        Rl=crearR(False)
        Rp=cargar('Rp'+str(usuario)+'V20.txt').reshape(12294,1).T
        return Rl,Rp

In [21]:
def recomendar(RpSelected,RSelected,d,idUser,minRating=10,genre=None,tipo=None,episodios=None,ranking=None,members=None):  
    recomendacion=np.array([],dtype='object').reshape(0,len(d[0]))  
    for i in tqdm(range(len(d))):
        if RpSelected[i]>=minRating and RSelected[0,i]!=-1:
            boolean=True
            if genre!=None and all(elem in genre for elem in d[i,2].split(',')):
                boolean=False
            if boolean and tipo!=None and all(elem in ['d[i,3]'] for elem in tipo ):
                boolean=False  
            if boolean and episodios!=None:
                try:
                    if not(episodios >= int(d[i,4])):
                        boolean=False
                except ValueError:
                        boolean=False
            if boolean and ranking!=None:
                try:
                    if not(ranking <= int(d[i,5])):
                        boolean=False
                except ValueError:
                        boolean=False  
            if boolean and members!=None:
                try:
                    if not(members <= int(d[i,5])):
                        boolean=False
                except ValueError:
                        boolean=False 
            if boolean:
                recomendacion=np.insert(recomendacion, recomendacion.shape[0], d[i], 0)                
    return recomendacion

def imprimirRecomendacion(recomendacion):
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
    pd.DataFrame(rd)

In [ ]:
def imprimirDiferencias(idUser,Rl,Rp,minR=0,minRp=0,maxR=10,maxRp=100):
    pos=0
    a=idUser-1
    b=0
    for i in range(len(Rl[0])-1):
        if Rl[a,i] != np.around(Rp[b,i]) and (Rl[a,i]>=minR and Rp[b,i]>=minRp and Rl[a,i]<=maxR and Rp[b,i]<=maxRp):
                print('UserAntes',Rl[a,i],'UserDespues',Rp[b,i])
                pos=pos+1
    print('Cantidad Diferencias: ',pos)
    
print(Rp)
imprimirDiferencias(67501,Rl,Rp,minR=1)

In [12]:
#HALLAR R PRIMA

usuario=51
kNeighbors=20
kFactorizacion=20

#Usuario
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)

print(Rp)
print(Rl)

[[10.00147724  9.25700474  8.27261639 ...  9.15034389  8.26238155
   8.67342186]]
[[ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [10  0  0 ...  0  8  0]
 [ 0  0  0 ...  0  0  0]]


4- Realice las siguiente recomendaciones (animes que no ha visto con calificación 10):

    1- Usuario de id 51, películas.
    2- Usuario de id 510, animes con menos de 31 episodios.
    3- Usuario de id 17000, animes con rankins mayores o iguales a 9.
    4- Usuario de id 27005, animes de comedia y aventura.
    5- Usuario de id 37502, TV.
    6- Usuario de id 57502, películas y OVAS.
    7- Usuario de id 67501.

Cada recomendación debe tener como respuesta, número de iteraciones, número de recomendaciones y nombre de los animes,

In [32]:
#Establecer Parametros

usuario=51
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
#Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
Rl = crearR(False)[usuario-1]
Rp = cargar("Rp51V20.txt")
recomendacion=recomendar(Rp,Rl.reshape((1, -1)),cargarAnimes(),usuario,minRating=10,tipo=['Movie'])
imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|██████████| 12294/12294 [00:00<00:00, 742986.85it/s]


In [ ]:
#Establecer Parametros

usuario=510
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
#Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
R = cargar("indices510.txt")
Rp = cargar("Rp510V20.txt")
recomendacion=recomendar(Rp,Rl,cargarAnimes(),usuario,minRating=10,episodios=31)
imprimirRecomendacion(recomendacion)
del Rl
del Rp

In [ ]:
#Establecer Parametros

usuario=17000
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
#Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
Rl = cargar("indices17000.txt")
Rp = cargar("Rp17000V20.txt")
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,ranking = 9)
imprimirRecomendacion(recomendacion)
del Rl
del Rp

In [19]:
#Establecer Parametros

usuario=27005
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
#Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
Rl = cargar("indices27005.txt")
Rp = cargar("Rp27005V20.txt")
recomendacion=recomendar(Rp,Rl.reshape((1, -1)),cargarAnimes(),usuario,minRating=10,genre = ['Comedy', 'Adventure'])
imprimirRecomendacion(recomendacion)
del Rl
del Rp

  0%|          | 0/12294 [00:00<?, ?it/s]


TypeError: 'in <string>' requires string as left operand, not list

In [ ]:
#Establecer Parametros

usuario=37502
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
#Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
Rl = cargar("indices37502.txt")
Rp = cargar("Rp37502V20.txt")
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10, tipo=['TV'])
imprimirRecomendacion(recomendacion)
del Rl

In [ ]:
#Establecer Parametros

usuario=57502
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
#Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
Rl = cargar("indices57502.txt")
Rp = cargar("Rp57502V20.txt")
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10, tipo=['Movie' , 'OVA'])
imprimirRecomendacion(recomendacion)
del Rl

In [ ]:
#Establecer Parametros

usuario=67501
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
#Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
Rl = cargar("indices67501.txt")
Rp = cargar("Rp67501V20.txt")
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10)
imprimirRecomendacion(recomendacion)
del Rl

In [38]:
#Establecer Parametros

usuario=510
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,episodios=31)
print(recomendacion)
imprimirRecomendacion(recomendacion)
del Rl
del Rp

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 821827.96it/s]

[[7311 'Suzumiya Haruhi no Shoushitsu'
  'Comedy, Mystery, Romance, School, Sci-Fi, Supernatural' 'Movie' '1'
  8.81 '240297']
 [12859 'One Piece Film: Z'
  'Action, Adventure, Comedy, Drama, Fantasy, Shounen' 'Movie' '1' 8.39
  '76051']
 [2167 'Clannad'
  'Comedy, Drama, Romance, School, Slice of Life, Supernatural' 'TV' '23'
  8.3 '566690']
 [5365 'Tsumiki no Ie' 'Drama' 'Movie' '1' 8.27 '45189']
 [4938 'Tsubasa: Shunraiki'
  'Action, Adventure, Drama, Fantasy, Magic, Mystery, Romance, Shounen, Supernatural'
  'OVA' '2' 8.23 '40420']
 [934 'Higurashi no Naku Koro ni'
  'Horror, Mystery, Psychological, Supernatural, Thriller' 'TV' '26' 8.17
  '359494']
 [6505 'There She Is!!' 'Comedy, Romance' 'ONA' '5' 8.11 '13935']
 [779 'Detective Conan Movie 01: The Timed Skyscraper'
  'Adventure, Comedy, Mystery, Police, Shounen' 'Movie' '1' 7.98 '28947']
 [875 'Mind Game' 'Adventure, Comedy, Dementia, Psychological, Romance'
  'Movie' '1' 7.88 '32756']
 [9074 'Arakawa Under the Bridge x Bridge' 

In [30]:
def transformarIndices(indices):
    R = crearR(False)
    RSelected = np.empty(cantAnimes)
    for i in range(0, len(indices)-1, 1):
        RSelected[i] = R[int(indices[i])]
    return Rselected